# Obtención de datos y conección a API

In [1]:
import requests
import pandas as pd

## Búsqueda de indicadores

In [37]:
url='http://api.worldbank.org/v2/es/indicator'
def consultar_indicadores(pagina=1):
    argumentos= {
        'page':pagina,
        'per_page':10000,
        'format':'json'
    }
    return requests.get(url,params=argumentos).json()

indicadores = consultar_indicadores()

# la primera parte de la respuesta 
# contiene información sobre las páginas
paginas = indicadores[0]["pages"]

# Reasigno la variable únicamente a los datos de mi interés 
# es una lista de diccionarios
indicadores = indicadores[1]

# recorro todás las páginas que tiene la respuesta 
# y lo voy anexando a la lista principal
for pagina in range(2, paginas+1):
    indicadores.extend(consultar_indicadores(pagina)[1])

In [38]:
indicators = pd.json_normalize(indicadores)

In [39]:
interes = input("Indicador de interés: ")
indicators[indicators.name.str.contains(interes,case=False)]

,id,name,unit,sourceNote,sourceOrganization,topics,source.id,source.value
17442,SP.DYN.LE00.FE.IN,"Esperanza de vida al nacer, mujeres (años)",,La esperanza de vida al nacer indica la cantid...,(1) División de Población de las Naciones Unid...,"[{'id': '8', 'value': 'Salud'}, {'id': '17', '...",2,Indicadores del desarrollo mundial
17443,SP.DYN.LE00.IN,"Esperanza de vida al nacer, total (años)",,La esperanza de vida al nacer indica la cantid...,Derived from male and female life expectancy a...,"[{'id': '8', 'value': 'Salud'}]",2,Indicadores del desarrollo mundial
17444,SP.DYN.LE00.MA.IN,"Esperanza de vida al nacer, varones (años)",,La esperanza de vida al nacer indica la cantid...,(1) División de Población de las Naciones Unid...,"[{'id': '8', 'value': 'Salud'}, {'id': '17', '...",2,Indicadores del desarrollo mundial


## Selección y extracción de un solo indicador

In [2]:
# Establecemos los parámetros que queremos acceder.
indicador = 'SP.DYN.LE00.IN'
pais = 'all'
iyear = '1980'
fyear = '2022'
page = '798'

# Página de la api y path al recurso solicitado
api_url = ' http://api.worldbank.org/v2'
path = f'/country/{pais}/indicator/{indicador}'

url = api_url + path

# Creamos el diccionario con los parametros 
# para el método get
def consultar(pagina = 1):
    args= {
        "date":iyear+":"+fyear,
        'page':pagina,
        "per_page":page,
        "format":"json",
        "prefix":"Getdata",
    }
    return requests.get(url,params=args).json()

dict_data = consultar()

In [3]:
# La primera parte de la respuesta nos indica en 
# cuantas páginas de encuentra la información
paginas=dict_data[0]["pages"]
paginas # Retornamos el número de páginas

14

In [4]:
# La segunda parte nos retorna una lista de diccionarios 
# con la información que queríamos
datos=dict_data[1]
datos

[{'indicator': {'id': 'SP.DYN.LE00.IN',
   'value': 'Life expectancy at birth, total (years)'},
  'country': {'id': 'ZH', 'value': 'Africa Eastern and Southern'},
  'countryiso3code': 'AFE',
  'date': '2021',
  'value': None,
  'unit': '',
  'obs_status': '',
  'decimal': 0},
 {'indicator': {'id': 'SP.DYN.LE00.IN',
   'value': 'Life expectancy at birth, total (years)'},
  'country': {'id': 'ZH', 'value': 'Africa Eastern and Southern'},
  'countryiso3code': 'AFE',
  'date': '2020',
  'value': 64.3257022314803,
  'unit': '',
  'obs_status': '',
  'decimal': 0},
 {'indicator': {'id': 'SP.DYN.LE00.IN',
   'value': 'Life expectancy at birth, total (years)'},
  'country': {'id': 'ZH', 'value': 'Africa Eastern and Southern'},
  'countryiso3code': 'AFE',
  'date': '2019',
  'value': 64.0052126245997,
  'unit': '',
  'obs_status': '',
  'decimal': 0},
 {'indicator': {'id': 'SP.DYN.LE00.IN',
   'value': 'Life expectancy at birth, total (years)'},
  'country': {'id': 'ZH', 'value': 'Africa Easter

In [5]:
# Agregamos los valores de las otras páginas a 
# nuestra lista de diccionarios
for pagina in range(2,paginas+1):
    datos.extend(consultar(pagina)[1])

In [6]:
# Verifico que lo obtenido sea una lista de diccionarios
print(type(datos))
print(type(datos[1]))

<class 'list'>
<class 'dict'>


In [7]:
# Mido el largo de la respuesta que es una lista
extención = len(datos)
extención

11172

In [8]:
# Creo el DataFrame con todos los datos
data = pd.json_normalize(datos)

In [9]:
# Creo un dataset para trabajar offline
#data.to_csv('../datasets/esperanza_de_vida.csv')

In [11]:
data.head()

,countryiso3code,date,value,unit,obs_status,decimal,indicator.id,indicator.value,country.id,country.value
0,AFE,2021,NaN,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",ZH,Africa Eastern and Southern
1,AFE,2020,64.325702,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",ZH,Africa Eastern and Southern
2,AFE,2019,64.005213,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",ZH,Africa Eastern and Southern
3,AFE,2018,63.648988,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",ZH,Africa Eastern and Southern
4,AFE,2017,63.246264,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",ZH,Africa Eastern and Southern


# EDA

In [14]:
data[data["country.value"].isna()]

,countryiso3code,date,value,unit,obs_status,decimal,indicator.id,indicator.value,country.id,country.value


In [15]:
datos=pd.read_csv('../datasets/esperanza_de_vida.csv')

In [16]:
datos[datos["country.value"].isna()]

,Unnamed: 0,countryiso3code,date,value,unit,obs_status,decimal,indicator.id,indicator.value,country.id,country.value
0,0,AFE,2021,NaN,NaN,NaN,0,SP.DYN.LE00.IN,"Esperanza de vida al nacer, total (años)",ZH,NaN
1,1,AFE,2020,64.325702,NaN,NaN,0,SP.DYN.LE00.IN,"Esperanza de vida al nacer, total (años)",ZH,NaN
2,2,AFE,2019,64.005213,NaN,NaN,0,SP.DYN.LE00.IN,"Esperanza de vida al nacer, total (años)",ZH,NaN
3,3,AFE,2018,63.648988,NaN,NaN,0,SP.DYN.LE00.IN,"Esperanza de vida al nacer, total (años)",ZH,NaN
4,4,AFE,2017,63.246264,NaN,NaN,0,SP.DYN.LE00.IN,"Esperanza de vida al nacer, total (años)",ZH,NaN
...,...,...,...,...,...,...,...,...,...,...,...
79,79,AFW,1984,47.830536,NaN,NaN,0,SP.DYN.LE00.IN,"Esperanza de vida al nacer, total (años)",ZI,NaN
80,80,AFW,1983,47.533985,NaN,NaN,0,SP.DYN.LE00.IN,"Esperanza de vida al nacer, total (años)",ZI,NaN
81,81,AFW,1982,47.188618,NaN,NaN,0,SP.DYN.LE00.IN,"Esperanza de vida al nacer, total (años)",ZI,NaN
82,82,AFW,1981,46.798292,NaN,NaN,0,SP.DYN.LE00.IN,"Esperanza de vida al nacer, total (años)",ZI,NaN
